Create .csv files for the PowerBI report.

In [2]:
import pandas as pd
import numpy as np

def normalize_special_chars(series):
    """Replace Hungarian special characters with ASCII equivalents, lowercase, and strip trailing spaces/underscores."""
    replacements = {
        'Á': 'A', 'á': 'a',
        'É': 'E', 'é': 'e',
        'Í': 'I', 'í': 'i',
        'Ó': 'O', 'ó': 'o',
        'Ö': 'O', 'ö': 'o',
        'Ő': 'O', 'ő': 'o',
        'Ú': 'U', 'ú': 'u',
        'Ü': 'U', 'ü': 'u',
        'Ű': 'U', 'ű': 'u',
        "'": ' ', '’': ' ',
        ' ': '_', '-': '_',
        '&': '_'
    }

    arr = np.array(series, dtype=str)
    for old, new in replacements.items():
        arr = np.char.replace(arr, old, new)
    
    # Lowercase
    arr = np.char.lower(arr)
    
    # Strip leading/trailing spaces or underscores
    arr = np.char.strip(arr, chars=" _")
    
    return arr


In [5]:
# Load data
df_transacts = pd.read_csv("transactions_raw_2025-09-27.csv")
#df_transacts = pd.read_csv("2025_07_transactions_raw.csv")

# Convert the 'date' column to datetime
df_transacts["date"] = pd.to_datetime(df_transacts["date"])

# Convert all object columns (except date) to string dtype
object_cols = df_transacts.select_dtypes(include="object").columns
df_transacts[object_cols] = df_transacts[object_cols].astype("string")

# Remove rows where df_transacts["value"] LIKE "%EUR"
# Then convert value column to integer
df_transacts = df_transacts[~df_transacts["value"].astype(str).str.contains("EUR", case=False, na=False)]
df_transacts["value"] = df_transacts["value"].astype(int)

# Remove rows where df_transacts["pay_method"] == "revolut"/"coupon_lidl"/"gift_card"
df_transacts = df_transacts[df_transacts["pay_method"].str.lower() != "revolut"]
df_transacts = df_transacts[df_transacts["pay_method"].str.lower() != "coupon_lidl"]
df_transacts = df_transacts[df_transacts["pay_method"].str.lower() != "gift_card"]

# Clean vendor names from spec. chars.
df_transacts["vendor_normalized"] = normalize_special_chars(df_transacts["vendor"])
df_transacts["vendor"] = df_transacts["vendor_normalized"]
df_transacts = df_transacts.drop(columns="vendor_normalized")


In [29]:
df_amend = df_transacts[
    (df_transacts["product_type"] == "monthly_income") |
    (df_transacts["product_type"] == "eat_out") |
    (df_transacts["product_type"] == "groceries") |
    (df_transacts["product_type"] == "fee") |
    (df_transacts["product_type"] == "body_care") |
    (df_transacts["product_type"] == "house_care") |
    (df_transacts["product_type"] == "travel")
]

In [30]:
np.unique(df_amend["product_type"])

array(['body_care', 'eat_out', 'fee', 'groceries', 'house_care',
       'monthly_income', 'travel'], dtype=object)

In [31]:
df_transacts.to_csv("transactions.csv", index=False)

In [32]:
# Create supplementary tables for

# Vendors
vendors_df = (
    pd.DataFrame(sorted(df_transacts["vendor"].dropna().unique()), columns=["vendor"])
)

# Dates
dates_df = (
    pd.DataFrame(sorted(df_transacts["date"].dropna().unique()), columns=["date"])
)

# Product types
product_types_df = (
    pd.DataFrame(sorted(df_transacts["product_type"].dropna().unique()), columns=["product_type"])
)

# Product names
product_names_df = (
    pd.DataFrame(sorted(df_transacts["product_name"].dropna().unique()), columns=["product_name"])
)

# Product names with product type (unique pairs)
# product_names_df = (
#     df_transacts[["product_type", "vendor", "product_name"]]
#     .dropna()
#     .drop_duplicates()
#     .sort_values(by=["product_type", "vendor", "product_name"])
#     .reset_index(drop=True)
# )

# Payment methods
pay_methods_df = (
    pd.DataFrame(sorted(df_transacts["pay_method"].dropna().unique()), columns=["pay_method"])
)

# Save to CSV
vendors_df.to_csv("vendors.csv", index=False)#, encoding="utf-8-sig")
dates_df.to_csv("dates.csv", index=False)#, encoding="utf-8-sig")
product_types_df.to_csv("product_types.csv", index=False)#, encoding="utf-8-sig")
product_names_df.to_csv("product_names.csv", index=False)#, encoding="utf-8-sig")
pay_methods_df.to_csv("pay_methods.csv", index=False)#, encoding="utf-8-sig")
